In [13]:
import pandas as pd
import plotly_express as px
from scipy import stats

In [14]:
trips_df = pd.read_csv('datasets/project_sql_result_01.csv')
dropoffs_df = pd.read_csv('datasets/project_sql_result_04.csv')
weather_df = pd.read_csv('datasets/project_sql_result_07.csv')

In [15]:
trips_df.info()
print(trips_df.duplicated().sum()) #to see if we have duplicated values
print(trips_df['company_name'].duplicated().sum()) #to show duplicates in case of a repeated company
trips_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
0
0


,company_name,trips_amount
0,Flash Cab,19558
1,Taxi Affiliation Services,11422
2,Medallion Leasin,10367
3,Yellow Cab,9888
4,Taxi Affiliation Service Yellow,9299
5,Chicago Carriage Cab Corp,9181
6,City Service,8448
7,Sun Taxi,7701
8,Star North Management LLC,7455
9,Blue Ribbon Taxi Association Inc.,5953


The trips_df table is clean, no need for cleaning process.

In [16]:
dropoffs_df.info()
print(dropoffs_df.duplicated().sum()) #to see if we have duplicated values
print(dropoffs_df['dropoff_location_name'].duplicated().sum()) #to show duplicates in case of a repeated company
dropoffs_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB
0
0


,dropoff_location_name,average_trips
0,Loop,10727.466667
1,River North,9523.666667
2,Streeterville,6664.666667
3,West Loop,5163.666667
4,O'Hare,2546.900000
5,Lake View,2420.966667
6,Grant Park,2068.533333
7,Museum Campus,1510.000000
8,Gold Coast,1364.233333
9,Sheffield & DePaul,1259.766667


The dropoffs_df table is clean, no need for cleaning process.

In [17]:
weather_df.info()
weather_df['start_ts'] = pd.to_datetime(weather_df['start_ts'], format= '%Y-%m-%d %H:%M:%S')
print(weather_df.duplicated().sum()) #to see if we have duplicated values
print(weather_df['start_ts'].duplicated().sum()) #to show duplicates in case of a repeated company
weather_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_seconds    1068 non-null   float64
dtypes: float64(1), object(2)
memory usage: 25.2+ KB
197
987


,start_ts,weather_conditions,duration_seconds
0,2017-11-25 16:00:00,Good,2410.0
1,2017-11-25 14:00:00,Good,1920.0
2,2017-11-25 12:00:00,Good,1543.0
3,2017-11-04 10:00:00,Good,2512.0
4,2017-11-11 07:00:00,Good,1440.0
5,2017-11-11 04:00:00,Good,1320.0
6,2017-11-04 16:00:00,Bad,2969.0
7,2017-11-18 11:00:00,Good,2280.0
8,2017-11-11 14:00:00,Good,2460.0
9,2017-11-11 12:00:00,Good,2040.0


In this table we are expecting to receive duplicates from column `start_ts` we will focus on change the type of the column by datetime64[ns] 

## Analysis

#### Neighborhoods

Let's show the top 10 neighborhoods in terms of average trips in drop-offs.

In [18]:
top10_dropoffs_df = dropoffs_df.sort_values(by='average_trips', ascending=False).head(10)
top10_dropoffs_df['average_trips'] = top10_dropoffs_df['average_trips'].round(2)
# top10_dropoffs_df

top10_dropoffs_df.describe() #average of 4324.988000

,average_trips
count,10.000000
mean,4324.988000
std,3530.185085
min,1259.770000
25%,1649.632500
50%,2483.935000
75%,6289.420000
max,10727.470000


In [19]:
fig1 = px.bar(top10_dropoffs_df, x='dropoff_location_name',y='average_trips', color=top10_dropoffs_df['dropoff_location_name'],
        title='TOP 10 DROPOFF NEIGHBORHOODS')

fig1.update_layout(xaxis_title='Location', yaxis_title='AVG Trips')
fig1.show()


According to this table, being 1259.8 the minimum and 10727.5 the maximum (in average of trips), Loop takes the lead, followed by River North, Streeterville and West Loop.

On a second category (below 4k average trips) we have O'hare, followed by Lake View and Grant Park.

Finally (below 2k average trips) Museum Campus, Goald Coast and Sheffield & DePaul are present in the respective order.

## Cab Companies

Well, now we know the best neighborhoods to dropoffs let's analize which cab companies are the most used

In [20]:
top10_cab_companies =trips_df.sort_values(by='trips_amount', ascending=False).head(10)
top10_cab_companies['market_share'] = 100 *(top10_cab_companies['trips_amount'] / top10_cab_companies['trips_amount'].sum())
top10_cab_companies # top10_cab_companies.describe()

,company_name,trips_amount,market_share
0,Flash Cab,19558,19.701426
1,Taxi Affiliation Services,11422,11.505762
2,Medallion Leasin,10367,10.443025
3,Yellow Cab,9888,9.960513
4,Taxi Affiliation Service Yellow,9299,9.367193
5,Chicago Carriage Cab Corp,9181,9.248328
6,City Service,8448,8.509952
7,Sun Taxi,7701,7.757474
8,Star North Management LLC,7455,7.509670
9,Blue Ribbon Taxi Association Inc.,5953,5.996656


In [21]:
fig2= px.bar(data_frame=top10_cab_companies, x='company_name', y='trips_amount', color='company_name' ,title='TOP 10 CAB COMPANIES')
fig2.update_layout(xaxis_title='Company Name', yaxis_title='Amount of Trips')

As we can see in the graphic below, `Flash cab` stands out from the rest of the companies, representing the 19.7% of market share (taking the top 10 as the total amount of trips).

'Taxi Affilitation Services' and 'Medallion Leasin' occupy the second and third place with an amount above from 10 thousand trips.

Finally below the 10 thousand and above the 5 thousand trips, the rest of the 10 cab companies have presence in the following order: 'Yellow Cab' , 'Taxi Affiliation Service Yellow', 'Chicago Carriage Cab Corb', 'City Service', 'Sun Taxi', 'Star North Management LCC' and 'Blue Ribbon Taxi Association Inc'.

## Hypothesis

The next dataframe conains data about the trips from Loop to the International Airport of O'Hare. 

`We will analyze the hypothesis 'The average lenght for the trips from Loop to The International Airport of O'Hare changes on rainy saturdays.'` = H<sub>A</sub>

In other words we can represent the following sentences.

H<sub>0</sub> = The average lenght of trips in 'Good' days is the same than 'Bad' days.

H<sub>A</sub> = The average lenght of trips in 'Good' days is the different than 'Bad' days.

First, we wil reduce our weather_df to just show us results on saturdays.

In [22]:
weather_df['dow'] = weather_df['start_ts'].dt.day_of_week
weather_df['dow'].unique()

array([5])

This dataframe already stores the data of saturdays only, now we will divide the table into two tables ('Good' and 'Bad' weather)

In [23]:
duration_good_days_df = weather_df[weather_df['weather_conditions']== 'Good'] ['duration_seconds']
duration_bad_days_df = weather_df[weather_df['weather_conditions']== 'Bad'] ['duration_seconds']

Let's proceed with the hypothesis. Because we have two dataframes to compare, we can use the stats.ttest_ind() method.

In [24]:
alpha = 0.05

t_statistic, p_value= stats.ttest_ind(duration_good_days_df,duration_bad_days_df, equal_var=False)

print(f'The t.statistic is :{t_statistic}')
print(f'The p.value is :{p_value}')

if p_value < alpha:
    print('The hypothesis "The average lenght of trips in \'Good\' days is the same than \'Bad\' days." has been rejected')
else:
    print('The hypothesis CAN NOT been rejected')

The t.statistic is :-7.186034288068629
The p.value is :6.738994326108734e-12
The hypothesis "The average lenght of trips in 'Good' days is the same than 'Bad' days." has been rejected


With this resulta, we conclude that we have enough statistical information to reject the hypothesis that ensures the average lenght of trips in Good days is the same than bad days.